## Data origin :: ane.energy

The data is we use in this notebook is given from the enterprise "ane.energy" based in Germany.

## Imports

### Modules, classes and functions

In [5]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
import json

from functools import reduce



# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

#windowfactor = 15

import warnings

## Part 1: Data mining

We import the data set, explore it briefly, drop duplicates and unused features and cast the data types.

## Functions

In [6]:
def import_csv(path):
    '''
    
    '''
    
    df = pd.read_csv(path, parse_dates=['dt_start_utc'], index_col='dt_start_utc')
    df.sort_index(inplace=True)
    
    return df 

In [7]:
def obj_2_datetime(df_list):
    '''
    The given list of multiple data frames transforms the "dt_start_utc" string-column to 
    a datetime column and set it to the index of each data frame 
    '''
    for df in df_list:
        df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"])
        df.sort_index(inplace=True)
    return df

## Import Data :: EPEX Day-Ahead Market

EPEX SPOT operates organized short-term electricity markets for Central Western Europe, Great Britain, the Nordics and Poland. 
 
EPEX Day-Ahead market is operated through a blind auction which takes place once a day, every day of the year. The traded electricity is delivered on the next day. Apart from Switzerland and Great Britain, all markets are part of the Multi-Regional Coupling (MRC) which stretches across 19 markets from Portugal to Finland and from The Netherlands to Italy. All auction markets of EPEX SPOT run on the ETS trading system, a robust and reliable trading platform. 

In [8]:
df_epex_da = import_csv("../data/ane_energy/epex_da_de.csv") 
df_epex_da.shape

(144911, 4)

In [9]:
df_epex_da.head()

,Unnamed: 0,sechs_h_regelung,epex_da_de_eur_mwh,epex_da_de_mwh
dt_start_utc,,,,
2004-12-31 23:00:00,0,0,23.89,NaN
2005-01-01 00:00:00,1,0,20.05,NaN
2005-01-01 01:00:00,2,0,15.00,NaN
2005-01-01 02:00:00,3,0,13.41,NaN
2005-01-01 03:00:00,4,0,13.73,NaN


In [17]:
df_epex_da.tail()

,Unnamed: 0,sechs_h_regelung,epex_da_de_eur_mwh,epex_da_de_mwh
dt_start_utc,,,,
2021-07-13 17:00:00,144906,0,105.00,NaN
2021-07-13 18:00:00,144907,0,100.29,NaN
2021-07-13 19:00:00,144908,0,98.52,NaN
2021-07-13 20:00:00,144909,0,95.03,NaN
2021-07-13 21:00:00,144910,0,86.10,NaN


In [16]:
print(df_epex_da["Unnamed: 0"].nunique())

144911


In [19]:
print(df_epex_da["epex_da_de_mwh"].unique())

[nan]


In [27]:
print((df_epex_da.sechs_h_regelung.sum()))

738


The column "Unnamed: 0" has 144911 unique values. It stars with 0 and ends with 144910. 
All missing values are in the column "epex_da_de_mwh". Looking at the shape, head, tail and 
number of unqiues of the data frame and compare to the the column "Unnamed:0", we can see that 
this column just counting up the number of rows. It's reasonable to remove the whole column 
from this data frame. "Sechs_h_regelung" isn't just filled with "0", so we decide to keep it.
We can conclude, that this column doesn't provide useful information. We drop this column 
as well as the column named "epex_da_de_mwh" with consist of only "NaN" values.

In [28]:
df_epex_da.drop(['Unnamed: 0', 'epex_da_de_mwh'], axis=1, inplace=True);

KeyError: "['Unnamed: 0' 'epex_da_de_mwh'] not found in axis"

In [22]:
print("The Data Frame has",df_epex_da.isnull().sum().sum(),"missing values. Period:", df_epex_da.index.min(), "till",df_epex_da.index.max())

The Data Frame has 0 missing values. Period: 2004-12-31 23:00:00 till 2021-07-13 21:00:00


The periods of this data frame is one hour. We will resample this data frame by 15 minutes, so it will fit our others data frame.

The EPEX Day Ahead bids for the auctions for the coming day must be placed by 12 noon. The results of the winning bids will be published at 12:40 p.m. each day.
In this case, we can use a polynominal interpolation without causing data leakage.

In [30]:
df_epex_da = df_epex_da.resample(resample_size).interpolate(method='polynomial', order=2)

We finally check, if any NaNs and missing values are in this data frame.

In [33]:
count_inf = np.isinf(df_epex_da).values.sum()
count_nan = df_epex_da.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 0 missing values.


## 1.2 Import ES Forcasts

In [34]:
def import_csv(path):
    
    df = pd.read_csv(path, parse_dates=['dt_start_utc'], index_col='dt_start_utc')
    df.sort_index(inplace=True)
    
    return df

In [75]:
def obj_2_datetime(df_list):
    for df in df_list:
        df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"])
        df.sort_index(inplace=True)
    return df

## Import:  Forcast Data

This data is preprocessed by the cooperation team ([Matthias](https://github.com/matthias-schmidt) & [Boris](https://github.com/Antadurunnu)).

In [35]:
df_es_fc_solar_ts = import_csv("../data/ane_energy/es_fc_solar_ts.csv") 
df_es_fc_total_load_ts = import_csv("../data/ane_energy/es_fc_total_load_ts.csv")
df_es_fc_total_renewables_ts = import_csv("../data/ane_energy/es_fc_total_renewables_ts.csv")
df_es_fc_wind_offshore_ts = import_csv("../data/ane_energy/es_fc_wind_offshore_ts.csv")
df_es_fc_wind_onshore_ts = import_csv("../data/ane_energy/es_fc_wind_onshore_ts.csv")

In [36]:
df_es_fc_solar_ts.name = "df_es_fc_solar_ts"
df_es_fc_total_load_ts.name = "df_es_fc_total_load_ts"
df_es_fc_total_renewables_ts.name = "df_es_fc_total_renewables_ts"
df_es_fc_wind_offshore_ts.name = "df_es_fc_wind_offshore_ts"
df_es_fc_wind_onshore_ts.name = "df_es_fc_wind_onshore_ts"

To streamline the data frame processing, we add the forcast data frames to a list.

In [39]:
df_list_es = [df_es_fc_solar_ts, 
                df_es_fc_total_load_ts, 
                df_es_fc_total_renewables_ts,
                df_es_fc_wind_offshore_ts,
                df_es_fc_wind_onshore_ts
          ]

We will print, the total period and check the sum of NaNs in each data frame

In [45]:
for df in df_list_es:
    print(df.head)
    print("The Data Frame",df.name,"has",df.isnull().sum().sum(),"missing values. Period:", df.index.min(), "till",df.index.max())

<bound method NDFrame.head of                      50Hertz_power_mw  DE_power_mw  DK_power_mw  DK1_power_mw  \
dt_start_utc                                                                    
2016-12-31 23:00:00               0.0          0.0          0.0           0.0   
2016-12-31 23:15:00               0.0          0.0          0.0           0.0   
2016-12-31 23:30:00               0.0          0.0          0.0           0.0   
2016-12-31 23:45:00               0.0          0.0          0.0           0.0   
2017-01-01 00:00:00               0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
2021-07-13 20:45:00               0.0          0.0          0.0           0.0   
2021-07-13 21:00:00               0.0          0.0          0.0           0.0   
2021-07-13 21:15:00               0.0          0.0          0.0           0.0   
2021-07-13 21:30:00               0.0          0.0          0.0           0.0  

We can see that all data frames are starting from 2016-12-31 23:00:00 to 2021-07-13 21:45:00 and they haven't any NaN's values.
We merge them to a single "forcast" dataframe

In [48]:
df_merged_es = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],
                                            how='inner'), df_list_es)

/tmp/ipykernel_348/3649181356.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'50Hertz_power_mw_x', 'DK_power_mw_x', 'DE_power_mw_x', 'TTG_power_mw_x', 'DK1_power_mw_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged_es = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],


In [49]:
df_merged_es.tail()

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw,DE_power_mw,DK_power_mw,DK1_power_mw,TTG_power_mw
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-07-13 20:45:00,0.0,0.0,0.0,0.0,0.0,9651.0,51324.0,3505.0,2268.0,14931.0,...,323.0,743.0,226.0,127.0,420.0,3637.0,8334.0,274.0,193.0,2620.0
2021-07-13 21:00:00,0.0,0.0,0.0,0.0,0.0,9421.0,49935.0,3289.0,2144.0,14327.0,...,330.0,747.0,214.0,120.0,417.0,3677.0,8426.0,297.0,208.0,2666.0
2021-07-13 21:15:00,0.0,0.0,0.0,0.0,0.0,9244.0,49336.0,3289.0,2144.0,14301.0,...,337.0,746.0,214.0,120.0,409.0,3580.0,8309.0,297.0,208.0,2678.0
2021-07-13 21:30:00,0.0,0.0,0.0,0.0,0.0,9065.0,48488.0,3289.0,2144.0,14043.0,...,343.0,746.0,214.0,120.0,403.0,3485.0,8192.0,297.0,208.0,2688.0
2021-07-13 21:45:00,0.0,0.0,0.0,0.0,0.0,8914.0,47817.0,3289.0,2144.0,13865.0,...,350.0,744.0,214.0,120.0,394.0,3371.0,8061.0,297.0,208.0,2699.0


## Final ane.energy Merge 

In [50]:
df_merged_3 = df_epex_da.merge(df_merged_es, left_index=True, right_index=True)

In [51]:
count_inf = np.isinf(df_merged_3).values.sum()
count_nan = df_merged_3.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 0 missing values.


In [52]:
df_merged_3.head()

,sechs_h_regelung,epex_da_de_eur_mwh,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw,DE_power_mw,DK_power_mw,DK1_power_mw,TTG_power_mw
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2016-12-31 23:00:00,0.000000e+00,20.960000,0.0,0.0,0.0,0.0,0.0,6291.0,40241.0,2942.0,...,331.0,3340.0,1004.0,652.0,3009.0,4922.0,12966.0,2282.0,1857.0,6636.0
2016-12-31 23:15:00,2.943790e-88,20.463647,0.0,0.0,0.0,0.0,0.0,6134.0,39805.0,2942.0,...,331.0,3338.0,1004.0,652.0,3007.0,4890.0,12916.0,2282.0,1857.0,6614.0
2016-12-31 23:30:00,2.667555e-88,20.622226,0.0,0.0,0.0,0.0,0.0,5976.0,39023.0,2942.0,...,331.0,3339.0,1004.0,652.0,3008.0,4859.0,12873.0,2282.0,1857.0,6595.0
2016-12-31 23:45:00,1.057543e-88,20.934692,0.0,0.0,0.0,0.0,0.0,5871.0,38682.0,2942.0,...,331.0,3339.0,1004.0,652.0,3008.0,4833.0,12830.0,2282.0,1857.0,6572.0
2017-01-01 00:00:00,0.000000e+00,20.900000,0.0,0.0,0.0,0.0,0.0,5810.0,38162.0,2735.0,...,331.0,3370.0,1094.0,748.0,3039.0,4808.0,12315.0,2152.0,1774.0,6078.0


## Save Data Set
For further work we save the data frame as a "pickle".

In [53]:
df_merged_3.to_pickle('../data/pickle/df_merged_3.pickle')